# 深入理解现有检索机制
---

## 1. 环境准备
- 安装依赖包
- 设置镜像源

In [2]:
# 设置国内镜像源（加速下载）
import os

from langchain_core.output_parsers import StrOutputParser

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# 安装依赖包（取消注释运行）
# !pip install langchain faiss-cpu huggingface-hub dashscope PyPDF2

## 2. 文档处理
- 加载PDF文件
- 检查加载结果

In [3]:
from langchain_community.document_loaders import PyPDFLoader

def load_documents(pdf_paths):
    """加载pdf文档并返回LangChain Document对象列表"""
    all_docs = []
    for path in pdf_paths:
        try:
            loader = PyPDFLoader(path)
            docs = loader.load()
            all_docs.extend(docs)
            print(f"成功加载：{path}（共{len(docs)}页）")
        except Exception as e:
            print(f"加载失败{path}：{str(e)}")
    return all_docs

# 示例使用
pdf_paths = ["pdf_China/中国产品质量法.pdf"]
processed_documents = load_documents(pdf_paths) # 原始文件
print(processed_documents)

成功加载：pdf_China/中国产品质量法.pdf（共10页）
[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-16T11:54:10+08:00', 'moddate': '2025-03-16T11:54:10+08:00', 'source': 'pdf_China/中国产品质量法.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='中华人民共和国产品质量法  \n \n（1993年2月22日第七届全国人民代表大会常务委员会第三十次会议通过  根据 2000年7\n月8日第九届全国人民代表大会常务委员会第十六次会议 《关于修改〈中华人民共和国产品\n质量法〉的决定》第一次修正   根据 2009年8月27日第十一届全国人民代表大会常务委员\n会第十次会议《关于修改部分法律的决定》第二次修正  根据 2018年12月29日第十三届全\n国人民代表大会常务委员会第七次会议《关于修改〈中华人民共和国产品质量法〉等五部法\n律的决定》第三次修正）  \n \n  目  录 \n \n  第一章  总  则 \n  第二章  产品质量的监督  \n  第三章  生产者、销售者的产品质量责任和义务  \n  第一节  生产者的产品质量责任和义务  \n  第二节  销售者的产品质量责任和义务  \n  第四章  损害赔偿  \n  第五章  罚  则 \n  第六章  附  则 \n \n  第一章  总  则 \n \n  第一条  为了加强对产品质量的监督管理，提高产品质量水平，明确产品质量责任，保\n护消费者的合法权益，维护社会经济秩序，制定本法。  \n \n  第二条  在中华人民共和国境内从事产品生产、销售活动，必须遵守本法。  \n \n  本法所称产品是指经过加工、制作，用于销售的产品。  \n \n  建设工程不适用本法规定；但是，建设工程使用的建筑材料、建筑构配件和设备，属于\n前款规定的产品范围的，适用本法规定。  \n \n  第三条  生产者、 销售者应当建立健全内部产品质量管

## 3. 文本分割与向量化

- 中文文本分割

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(docs):
    """执行文本分割"""
    if not docs:
        raise ValueError("输入文档列表为空")

    # 中文优化分割器
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,     # 每个文本块500字符
        chunk_overlap=100,  # 块间重叠100字符
        separators=["\n\n", "\n", "。", "！", "？"] # 中文分隔符
    )

    # 执行分割
    split_docs = text_splitter.split_documents(docs)
    print(f"分割为{len(split_docs)}个文本块")

    # 查看前2个分割样例
    for i, doc in enumerate(split_docs[:2]):
        print(f"\n块{i+1}:\n{doc.page_content[:200]}...")

    return split_docs

# 示例使用
split_decs = split_documents(processed_documents)

分割为26个文本块

块1:
中华人民共和国产品质量法  
 
（1993年2月22日第七届全国人民代表大会常务委员会第三十次会议通过  根据 2000年7
月8日第九届全国人民代表大会常务委员会第十六次会议 《关于修改〈中华人民共和国产品
质量法〉的决定》第一次修正   根据 2009年8月27日第十一届全国人民代表大会常务委员
会第十次会议《关于修改部分法律的决定》第二次修正  根据 2018年12月29日第十三届全
国人...

块2:
第一章  总  则 
 
  第一条  为了加强对产品质量的监督管理，提高产品质量水平，明确产品质量责任，保
护消费者的合法权益，维护社会经济秩序，制定本法。  
 
  第二条  在中华人民共和国境内从事产品生产、销售活动，必须遵守本法。  
 
  本法所称产品是指经过加工、制作，用于销售的产品。  
 
  建设工程不适用本法规定；但是，建设工程使用的建筑材料、建筑构配件和设备，属于
前款规...


- 构建向量数据库

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

def create_vector_store(split_docs):
    """创建并返回向量数据库"""
    if not split_docs:
        raise ValueError("无法用空文档创建向量库")

    embeddings = HuggingFaceEmbeddings(
        model_name="distiluse-base-multilingual-cased-v2"
    )

    vector_store = FAISS.from_documents(split_docs, embeddings)
    print(f"向量库已构建（包含{vector_store.index.ntotal}）个向量")

    return vector_store

# 示例使用
vector_store = create_vector_store(split_decs)

D:\files\Anaconda\envs\env_rag_develop\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


向量库已构建（包含26）个向量


- 检查向量数据库中的基本信息

In [6]:
print(f"向量数据库类型：{type(vector_store)}")

print(f"向量数据库中文档数量：{vector_store.index.ntotal}")

print(f"向量维度：{vector_store.index.d}")

向量数据库类型：<class 'langchain_community.vectorstores.faiss.FAISS'>
向量数据库中文档数量：26
向量维度：512


## 4. 分析当前检索系统的组成

- **向量检索**：目前使用的是FAISS + HuggingFace嵌入模型(distiluse-base-multilingual-cased-v2)
- **文本分割**：RecursiveCharacterTextSplitter以500字符为单位进行分割
- **检索参数**：top-k=3的相似度检索

## 5. 基础实验与性能评估

In [7]:
import numpy as np
# 检索分析实验
def analyze_retrieval(query, vector_store, top_k=3):
    """深入分析实验结果"""
    # 获取原始检索结果
    docs = vector_store.similarity_search(query, k=top_k)

    # 计算相似度分数
    embeddings = vector_store.embedding_function # 返回嵌入模型对象（vector_store没有其他用）
    query_embedding = embeddings.embed_query(query) # 调用嵌入模型对问题编码

    print(query_embedding) # 输出问题的向量

    print(f"\n查询：'{query}'")
    print(f"检索到的{len(docs)}个文档：")

    for i, doc in enumerate(docs[:top_k]):
        doc_embedding = embeddings.embed_query(doc.page_content) # 调用嵌入模型对检索到的文本编码
        similarity = np.dot(query_embedding, doc_embedding) # 计算查询向量和文档向量之间的余弦相似度
        print(f"\n📄 文档{i+1} (相似度: {similarity:.4f}):")
        print(f"  来源: {doc.metadata.get('source', '未知')}")
        print(f"  内容: {doc.page_content[:50]}...")

# 示例使用
queries = [
    "产品质量监督抽查的规定",
    "产品缺陷的法律定义",
    "违反产品质量法的处罚措施"
]

for query in queries:
    analyze_retrieval(query, vector_store)


[0.05332158878445625, 0.04242091253399849, 0.0725243017077446, 0.01012426521629095, 0.009805211797356606, 0.012592888437211514, 0.02883034758269787, -3.078673034906387e-05, -0.033212222158908844, 0.020445043221116066, -0.006073226686567068, 0.010134180076420307, 0.007285805884748697, -0.020407484844326973, 0.04960126429796219, 0.0143410200253129, 0.029170896857976913, 0.06669554114341736, -0.003052848158404231, 0.03200319781899452, 0.04146166145801544, -0.027783019468188286, -0.0704202800989151, -0.03058229200541973, 0.0665045753121376, -0.07274922728538513, 0.06031861528754234, 0.0027812179177999496, 0.014749638736248016, 0.037064243108034134, -0.00887268502265215, -0.02358645759522915, -0.0005606429767794907, 0.02669362910091877, 0.01654990017414093, -0.014764975756406784, 0.009736333042383194, 0.02399057149887085, -0.08841714262962341, -0.0322066955268383, 0.011998040601611137, -0.01991821452975273, -0.11747483164072037, -0.01829858310520649, 0.03147347271442413, 0.03068734146654606

## 6. 检索质量评估指标

**建立简单的评估体系**：
- 召回率(Recall)：相关文档被检索到的比例
- 准确率(Precision)：检索结果中相关文档的比例
- 平均倒数排名(MRR)：第一个相关文档排名的倒数

In [8]:
# 简易评估框架
def evaluate_retrieval(query, relevant_doc_indices, vector_store, top_k=3):
    """评估单个查询的检索结果"""
    # 检索到的文档信息
    docs = vector_store.similarity_search(query, k=top_k)
    # 提取检索到的页码
    retrieved_indices = [doc.metadata.get('page', -1) for doc in docs]

    print("docs类型：", type(docs))
    # print(docs)

    print("retrieved_indices类型：", type(retrieved_indices))
    print("检索到的文档页码：", retrieved_indices)

    # 计算指标
    # 检索到的又相关的 = 检索到的 & 相关的
    relevant_retrieved = len(set(retrieved_indices) & set(relevant_doc_indices))

    # 准确率 = 检索到的相关文档数/检索到的文档总数
    precision = relevant_retrieved / top_k

    # 召回率 = 检索到的相关文档数 / 所有相关文档数
    recall = relevant_retrieved / len(relevant_doc_indices)

    # 平均倒数排名（在检索到的文档中，第一个相关文档排名的倒数）（省略了多次查询求平均的步骤）
    try:
        first_relevant_doc = min([i+1 for i, idx in enumerate(retrieved_indices)
                                  if idx in relevant_doc_indices])
        mrr = 1 / first_relevant_doc
    except :
        mrr = 0

    # 返回一个字典
    return {
        'query': query,
        'precision': precision,
        'recall': recall,
        'mrr': mrr
    }

# 示例评估
evaluation_results = []
evaluation_results.append(evaluate_retrieval(
    "产品质量监督抽查的规定",
    [1, 2, 3], # 假设这些页码包含相关内容
    vector_store
))
print(evaluation_results)

docs类型： <class 'list'>
[Document(id='ed0878d0-40a9-4c60-9cb2-4b946e30f82c', metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-16T11:54:10+08:00', 'moddate': '2025-03-16T11:54:10+08:00', 'source': 'pdf_China/中国产品质量法.pdf', 'total_pages': 10, 'page': 2, 'page_label': '3'}, page_content='国家监督抽查的产品，地方不得另行重复抽查；上级监督抽查的产品，下级不得另行重\n复抽查。  \n \n  根据监督抽查的需要，可以对产品进行检验。检验抽取样品的数量不得超过检验的合理\n需要，并不得向被检查人收取检验费用。监督抽查所需检验费用按照国务院规定列支。  \n \n  生产者、销售者对抽查检验的结果有异议的，可以自收到检验结果之日起十五日内向实\n施监督抽查的市场监督管理部门或者其上级市场监督管理部门申请复检， 由受理复检的市场\n监督管理部门作出复检结论。  \n \n  第十六条  对依法进行的产品质量监督检查，生产者、销售者不得拒绝。  \n \n  第十七条  依照本法规定进行监督抽查的产品质量不合格的，由实施监督抽查的市场监\n督管理部门责令其生产者、销售者限期改正。逾期不改正的，由省级以上人民政府市场监督\n管理部门予以公告；公告后经复查仍不合格的，责令停业，限期整顿；整顿期满后经复查产\n品质量仍不合格的，吊销营业执照。  \n \n  监督抽查的产品有严重质量问题的，依照本法第五章的有关规定处罚。'), Document(id='43aa78a7-c29d-4cce-91c0-af35171150a8', metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-03-16T11:54:10+08:00', 'moddate'